# Github Repo
https://github.com/lucalin17081994/Syntax-Enhanced-Bert

# Imports + GPU setup

In [ ]:
!pip install transformers
!apt-get install git
!git clone https://github.com/lucalin17081994/Syntax-Enhanced-Bert/
%cd Syntax-Enhanced-Bert
!git pull
# !pip install pandas --upgrade

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.25.1-1ubuntu3.11).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Cloning into 'Syntax-Enhanced-Bert'...
remote: Enumerating objects: 1073, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 1073 (delta 134), reused 202 (delta 114), pack-reused 838
Receiving objects: 100% (1073/1073), 625.63 MiB | 23.77 MiB/s, done.
Resolving deltas: 100% (653/653), done.
Updating files: 100% (34/34), done.
Error downloading object: glove.6B.100d.txt (95dde4d): Smudge error: Error downloading glove.6B.100d.txt (95dde4dfd627ab26608d33e76d1195ec059734bd29089ea52cadb08d07c64544): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/Syntax-Enhanced-Bert/Syntax-Enhanced-Bert

In [ ]:
import Modules
from Modules import CA_Hesyfu, initialize_model, WarmupLinearSchedule
import Data
from Data import (
    read_dropna_encode_dataframe,
    read_data_pandas_snli,
    read_data_pandas,
    SNLI_Dataset,
    get_batch_sup,
)

import Evaluation
from Evaluation import log_eval_metrics,train_batch,log_eval_metrics
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import numpy as np
import pandas as pd
import time
import re
from sklearn import preprocessing
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from typing import Tuple, List, Callable, Optional
from torch import FloatTensor, LongTensor


In [ ]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
if torch.cuda.is_available():
   print("Training on GPU")
   device = torch.device("cuda:0")
else:
  device=torch.device("cpu")

# Run Pipeline

## Unzip file from drive

In [ ]:
from zipfile import ZipFile

file_name = "SNLI_2validations.zip"
with ZipFile(file_name, 'r') as zipfile:
    zipfile.printdir()
    zipfile.extractall()



File Name                                             Modified             Size
SNLI_val_hard.pickle                           2023-02-09 14:08:20      2838297
SNLI_train.pickle                              2023-02-08 19:31:44    471795666
SNLI_val.pickle                                2023-02-08 19:39:56      9090645


In [ ]:
pd.read_pickle('HELP_train.pickle')

,Unnamed: 0,filename,determiner,monotonicity,gold_label,replace_target,replace_source,replace_mode,ori_sentence,new_sentence,...,text_sentence1,text_sentence2,pos_sentence1,pos_sentence2,heads_sentence1,heads_sentence2,deprel_sentence1,deprel_sentence2,sentence1_parse,sentence2_parse
0,0,silver/p75/d3454,neither,downward_monotone,entailment,neither one of his parents,neither parents,simple,Tom said that neither parents had ever been to...,Tom said that neither one of his parents had e...,...,"[Tom, said, that, neither, parents, had, ever,...","[Tom, said, that, neither, one, of, his, paren...","[NNP, VBD, IN, DT, NNS, VBD, RB, VBN, IN, NNP, .]","[NNP, VBD, IN, DT, CD, IN, PRP$, NNS, VBD, RB,...","[2, 0, 10, 5, 10, 10, 10, 10, 10, 2, 2]","[2, 0, 13, 5, 13, 8, 8, 5, 13, 13, 13, 13, 2, 2]","[nsubj, root, mark, det, nsubj, aux, advmod, c...","[nsubj, root, mark, det, nsubj, case, nmod:pos...",(ROOT (S (NP (NNP Tom)) (VP (VBD said) (SBAR (...,(ROOT (S (NP (NNP Tom)) (VP (VBD said) (SBAR (...
1,1,silver/p14/d0959,neither,downward_monotone,entailment,neither of them,neither them,simple,I know neither them .,I know neither of them .,...,"[I, know, neither, them, .]","[I, know, neither, of, them, .]","[PRP, VBP, CC, PRP, .]","[PRP, VBP, DT, IN, PRP, .]","[2, 0, 4, 2, 2]","[2, 0, 2, 5, 3, 2]","[nsubj, root, det:predet, obj, punct]","[nsubj, root, obj, case, nmod, punct]",(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (CC n...,(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...
2,2,silver/p24/d2663,neither,downward_monotone,entailment,neither of his brothers,neither brothers,simple,I know neither brothers .,I know neither of his brothers .,...,"[I, know, neither, brothers, .]","[I, know, neither, of, his, brothers, .]","[PRP, VBP, DT, NNS, .]","[PRP, VBP, DT, IN, PRP$, NNS, .]","[2, 0, 4, 2, 2]","[2, 0, 2, 6, 6, 3, 2]","[nsubj, root, det, obj, punct]","[nsubj, root, obj, case, nmod:poss, nmod, punct]",(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (DT n...,(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...
3,3,silver/p24/d2663,neither,downward_monotone,neutral,brothers,friends,noun_hypernym_obj,I know neither of his brothers .,I know neither of his friends .,...,"[I, know, neither, of, his, brothers, .]","[I, know, neither, of, his, friends, .]","[PRP, VBP, DT, IN, PRP$, NNS, .]","[PRP, VBP, DT, IN, PRP$, NNS, .]","[2, 0, 2, 6, 6, 3, 2]","[2, 0, 2, 6, 6, 3, 2]","[nsubj, root, obj, case, nmod:poss, nmod, punct]","[nsubj, root, obj, case, nmod:poss, nmod, punct]",(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...,(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...
4,4,silver/p24/d2663,neither,downward_monotone,entailment,brothers,friends,noun_hypernym_obj,I know neither of his friends .,I know neither of his brothers .,...,"[I, know, neither, of, his, friends, .]","[I, know, neither, of, his, brothers, .]","[PRP, VBP, DT, IN, PRP$, NNS, .]","[PRP, VBP, DT, IN, PRP$, NNS, .]","[2, 0, 2, 6, 6, 3, 2]","[2, 0, 2, 6, 6, 3, 2]","[nsubj, root, obj, case, nmod:poss, nmod, punct]","[nsubj, root, obj, case, nmod:poss, nmod, punct]",(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...,(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35886,431,silver/p97/d2664,or,disjunction,entailment,NaN,NaN,NaN,There were too many sights to see in a day,There were too many sights to see in a day or ...,...,"[There, were, too, many, sights, to, see, in, ...","[There, were, too, many, sights, to, see, in, ...","[EX, VBD, RB, JJ, NNS, TO, VB, IN, DT, NN]","[EX, VBD, RB, JJ, NNS, TO, VB, IN, DT, NN, CC,...","[2, 0, 4, 5, 2, 7, 5, 10, 10, 7]","[2, 0, 4, 5, 2, 7, 5, 10, 10, 7, 12, 10, 2]","[expl, root, advmod, amod, nsubj, mark, acl, c...","[expl, root, advmod, amod, nsubj, mark, acl, c...",(ROOT (S (NP (EX There)) (VP (VBD were) (NP (N...,(ROOT (S (NP (EX There)) (VP (VBD were) (NP (N...
35887,432,silver/p97/d2794,or,disjunction,neutral,NaN,NaN,NaN,There are two or three pens on the desk .,There are two

In [ ]:
# import pickle
# #hardcoding to ensure every vocab is the same
# also avoids parsing entire training set again.
dep_lb_to_idx= {'det': 0, 'nsubj': 1, 'case': 2, 'nmod': 3, 'root': 4, 'amod': 5, 'compound:prt': 6, 'obl': 7, 'punct': 8, 'aux': 9, 'nmod:poss': 10, 'obj': 11, 'cop': 12, 'advcl': 13, 'cc': 14, 'conj': 15, 'expl': 16, 'xcomp': 17, 'compound': 18, 'mark': 19, 'nummod': 20, 'advmod': 21, 'acl': 22, 'acl:relcl': 23, 'cc:preconj': 24, 'fixed': 25, 'nsubj:pass': 26, 'aux:pass': 27, 'ccomp': 28, 'appos': 29, 'iobj': 30, 'det:predet': 31, 'flat': 32, 'parataxis': 33, 'obl:tmod': 34, 'obl:agent': 35, 'discourse': 36, 'nmod:npmod': 37, 'goeswith': 38, 'csubj': 39, 'list': 40, 'obl:npmod': 41, 'dep': 42, 'reparandum': 43, 'nmod:tmod': 44, 'orphan': 45, 'dislocated': 46, 'vocative': 47, 'csubj:pass': 48}
c_c_to_idx= {'NP': 0, 'DT': 1, 'NN': 2, 'PP': 3, 'IN': 4, 'S': 5, 'VP': 6, 'VBZ': 7, 'ADJP': 8, 'VBN': 9, 'RP': 10, '.': 11, 'VBG': 12, 'PRP$': 13, ',': 14, 'ADVP': 15, 'RB': 16, 'NNS': 17, 'CC': 18, 'PRP': 19, 'VBP': 20, 'EX': 21, 'JJ': 22, 'JJR': 23, 'SBAR': 24, 'TO': 25, 'VB': 26, 'CD': 27, 'HYPH': 28, 'WHNP': 29, 'WP': 30, 'VBD': 31, 'PRT': 32, 'NML': 33, 'NNP': 34, 'WHADVP': 35, 'WRB': 36, 'WDT': 37, 'POS': 38, 'QP': 39, 'WHADJP': 40, 'MD': 41, '``': 42, 'NNPS': 43, "''": 44, 'PDT': 45, 'RBR': 46, 'JJS': 47, 'UH': 48, 'UCP': 49, 'WHPP': 50, 'AFX': 51, 'SYM': 52, 'SINV': 53, 'X': 54, 'CONJP': 55, 'NFP': 56, 'FRAG': 57, 'WP$': 58, ':': 59, '-LRB-': 60, '-RRB-': 61, 'SBARQ': 62, 'SQ': 63, 'FW': 64, 'PRN': 65, 'INTJ': 66, 'RBS': 67, '$': 68, 'ADD': 69, 'RRC': 70, 'LST': 71, 'LS': 72, 'GW': 73}
w_c_to_idx= {'S': 0, 'NP': 1, 'DT': 2, 'NN': 3, 'PP': 4, 'IN': 5, 'VP': 6, 'VBZ': 7, 'ADJP': 8, 'VBN': 9, 'RP': 10, '.': 11, 'VBG': 12, 'PRP$': 13, ',': 14, 'ADVP': 15, 'RB': 16, 'NNS': 17, 'CC': 18, 'PRP': 19, 'VBP': 20, 'EX': 21, 'JJ': 22, 'JJR': 23, 'SBAR': 24, 'TO': 25, 'VB': 26, 'CD': 27, 'HYPH': 28, 'WHNP': 29, 'WP': 30, 'VBD': 31, 'PRT': 32, 'NML': 33, 'NNP': 34, 'WHADVP': 35, 'WRB': 36, 'WDT': 37, 'POS': 38, 'QP': 39, 'WHADJP': 40, 'MD': 41, '``': 42, 'NNPS': 43, "''": 44, 'PDT': 45, 'RBR': 46, 'JJS': 47, 'UH': 48, 'UCP': 49, 'WHPP': 50, 'AFX': 51, 'SYM': 52, 'SINV': 53, 'X': 54, 'CONJP': 55, 'NFP': 56, 'FRAG': 57, 'WP$': 58, ':': 59, '-LRB-': 60, '-RRB-': 61, 'SBARQ': 62, 'SQ': 63, 'FW': 64, 'PRN': 65, 'INTJ': 66, 'RBS': 67, '$': 68, 'ADD': 69, 'RRC': 70, 'LST': 71, 'LS': 72, 'GW': 73}

#mnli
# dep_lb_to_idx={'det': 0, 'nsubj': 1, 'case': 2, 'nmod': 3, 'root': 4, 'amod': 5, 'compound:prt': 6, 'obl': 7, 'punct': 8, 'aux': 9, 'nmod:poss': 10, 'obj': 11, 'cop': 12, 'advcl': 13, 'cc': 14, 'conj': 15, 'expl': 16, 'xcomp': 17, 'compound': 18, 'mark': 19, 'nummod': 20, 'advmod': 21, 'acl': 22, 'acl:relcl': 23, 'cc:preconj': 24, 'fixed': 25, 'nsubj:pass': 26, 'aux:pass': 27, 'ccomp': 28, 'appos': 29, 'iobj': 30, 'det:predet': 31, 'flat': 32, 'parataxis': 33, 'obl:tmod': 34, 'obl:agent': 35, 'discourse': 36, 'nmod:npmod': 37, 'goeswith': 38, 'csubj': 39, 'list': 40, 'obl:npmod': 41, 'dep': 42, 'reparandum': 43, 'nmod:tmod': 44, 'orphan': 45, 'dislocated': 46, 'vocative': 47, 'csubj:pass': 48, 'nsubj:outer': 49, 'advcl:relcl': 50, 'csubj:outer': 51}


In [ ]:
# check sentences in ANLI
file_name = "anli_train.zip"
with ZipFile(file_name, 'r') as zipfile:
    zipfile.printdir()
    zipfile.extractall()


File Name                                             Modified             Size
anli_train.pickle                              2023-06-04 20:44:48    354491804


In [ ]:
le = preprocessing.LabelEncoder()
le.fit(['contradiction', 'entailment', 'neutral']) #hardcode so you know the encoding in another notebook

train_data = read_dropna_encode_dataframe('anli_train.pickle',le,False)
train_data.head(100)

,gold_label,text_sentence1,text_sentence2,heads_sentence1,heads_sentence2,deprel_sentence1,deprel_sentence2,sentence1_parse,sentence2_parse
0,"[0, 1, 0]","[The, Parma, trolleybus, system, (, Italian, :...","[The, trolleybus, system, has, over, 2, urban,...","[4, 4, 4, 15, 9, 4, 6, 9, 4, 9, 9, 9, 9, 9, 0,...","[3, 3, 4, 0, 6, 8, 8, 4]","[det, compound, compound, nsubj, punct, appos,...","[det, compound, nsubj, root, advmod, nummod, a...",(ROOT (S (NP (NP (DT The) (NNP Parma) (NN trol...,(ROOT (S (NP (DT The) (NN trolleybus) (NN syst...
1,"[0, 0, 1]","[Alexandra, Lendon, Bastedo, (, 9, March, 1946...","[Sharron, Macready, was, a, popular, character...","[16, 1, 1, 5, 1, 5, 5, 9, 5, 9, 9, 5, 16, 16, ...","[6, 1, 6, 6, 6, 0, 9, 9, 6, 6]","[nsubj, flat, flat, punct, nmod:tmod, compound...","[nsubj, flat, cop, det, amod, root, case, det,...",(ROOT (S (NP (NP (NNP Alexandra) (NNP Lendon) ...,(ROOT (S (NP (NNP Sharron) (NNP Macready)) (VP...
2,"[0, 0, 1]","[Alexandra, Lendon, Bastedo, (, 9, March, 1946...","[Bastedo, did, n't, keep, any, pets, because, ...","[16, 1, 1, 5, 1, 5, 5, 9, 5, 9, 9, 5, 16, 16, ...","[4, 4, 4, 0, 6, 4, 10, 7, 10, 4, 13, 13, 10, 4]","[nsubj, flat, flat, punct, nmod:tmod, compound...","[nsubj, aux, advmod, root, det, obj, case, fix...",(ROOT (S (NP (NP (NNP Alexandra) (NNP Lendon) ...,(ROOT (S (NP (NNP Bastedo)) (VP (VBD did) (RB ...
3,"[0, 0, 1]","[Alexandra, Lendon, Bastedo, (, 9, March, 1946...","[Alexandra, Bastedo, was, named, by, her, moth...","[16, 1, 1, 5, 1, 5, 5, 9, 5, 9, 9, 5, 16, 16, ...","[4, 1, 4, 0, 7, 7, 4, 4]","[nsubj, flat, flat, punct, nmod:tmod, compound...","[nsubj:pass, flat, aux:pass, root, case, nmod:...",(ROOT (S (NP (NP (NNP Alexandra) (NNP Lendon) ...,(ROOT (S (NP (NNP Alexandra) (NNP Bastedo)) (V...
4,"[0, 0, 1]","[Alexandra, Lendon, Bastedo, (, 9, March, 1946...","[Bastedo, cared, for, all, the, animals, that,...","[16, 1, 1, 5, 1, 5, 5, 9, 5, 9, 9, 5, 16, 16, ...","[2, 0, 6, 6, 6, 2, 8, 6, 10, 8, 2]","[nsubj, flat, flat, punct, nmod:tmod, compound...","[nsubj, root, case, det:predet, det, obl, nsub...",(ROOT (S (NP (NP (NNP Alexandra) (NNP Lendon) ...,(ROOT (S (NP (NNP Bastedo)) (VP (VBD cared) (P...
...,...,...,...,...,...,...,...,...,...
95,"[1, 0, 0]","[The, Great, Mall, of, the, Bay, Area, (, ofte...","[The, Great, Mall, of, the, Bay, Are, was, bui...","[3, 3, 22, 7, 7, 7, 3, 11, 11, 11, 3, 14, 14, ...","[3, 3, 9, 7, 6, 3, 3, 9, 0, 11, 9]","[det, compound, nsubj, case, det, compound, nm...","[det, amod, nsubj:pass, case, det, nmod, nmod,...",(ROOT (S (NP (NP (NP (DT The) (NNP Great) (NNP...,(ROOT (S (NP (NP (DT The) (NNP Great) (NNP Mal...
96,"[1, 0, 0]","[The, Great, Mall, of, the, Bay, Area, (, ofte...","[The, Great, Mall, was, bought, by, the, Simon...","[3, 3, 22, 7, 7, 7, 3, 11, 11, 11, 3, 14, 14, ...","[3, 3, 5, 5, 0, 10, 10, 10, 10, 5, 12, 5]","[det, compound, nsubj, case, det, compound, nm...","[det, compound, nsubj:pass, aux:pass, root, ca...",(ROOT (S (NP (NP (NP (DT The) (NNP Great) (NNP...,(ROOT (S (NP (DT The) (NNP Great) (NNP Mall)) ...
97,"[1, 0, 0]","[The, Great, Mall, of, the, Bay, Area, (, ofte...","[The, Great, Mall, is, an, outdoor, shopping, ...","[3, 3, 22, 7, 7, 7, 3, 11, 11, 11, 3, 14, 14, ...","[3, 3, 8, 8, 8, 8, 8, 0]","[det, compound, nsubj, case, det, compound, nm...","[det, compound, nsubj, cop, det, amod, compoun...",(ROOT (S (NP (NP (NP (DT The) (NNP Great) (NNP...,(ROOT (S (NP (DT The) (NNP Great) (NNP Mall)) ...
98,"[0, 1, 0]","[Love, in, Nepal, is, a, 2004, Indian, Hindi, ...","[Sonu, Nigam, was, a, male, lead, in, a, film,...","[11, 3, 1, 11, 11, 11, 11, 11, 11, 11, 0, 11, ...","[6, 1, 6, 6, 6, 0, 9, 9, 6, 11, 9, 13, 11, 6]","[nsubj, case, nmod, cop, det, compound, amod, ...","[nsubj, flat, cop, det, amod, root, case, det,...",(ROOT (S (S (NP (NP (NNP Love)) (PP (IN in) (N...,(ROOT (S (NP (NNP Sonu) (NNP Nigam)) (VP (VBD ...


In [ ]:
le = preprocessing.LabelEncoder()
le.fit(['contradiction', 'entailment', 'neutral']) #hardcode so you know the encoding in another notebook

train_data = read_dropna_encode_dataframe('SNLI_train.pickle',le,False)
dev_data = read_dropna_encode_dataframe('SNLI_val.pickle',le,False)

train_data=train_data.drop(['sentence1', 'sentence2', 'pos_sentence1', 'pos_sentence2'],axis=1)
dev_data=dev_data.drop(['sentence1', 'sentence2', 'pos_sentence1', 'pos_sentence2'],axis=1)




In [ ]:
train_data

,gold_label,text_sentence1,text_sentence2,heads_sentence1,heads_sentence2,deprel_sentence1,deprel_sentence2,sentence1_parse,sentence2_parse
0,"[0, 0, 1]","[A, person, on, a, horse, jumps, over, a, brok...","[A, person, is, training, his, horse, for, a, ...","[2, 6, 5, 5, 2, 0, 11, 11, 11, 9, 6, 6]","[2, 4, 4, 0, 6, 4, 9, 9, 4, 4]","[det, nsubj, case, det, nmod, root, case, det,...","[det, nsubj, aux, root, nmod:poss, obj, case, ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,"[1, 0, 0]","[A, person, on, a, horse, jumps, over, a, brok...","[A, person, is, at, a, diner, ,, ordering, an,...","[2, 6, 5, 5, 2, 0, 11, 11, 11, 9, 6, 6]","[2, 6, 6, 6, 6, 0, 8, 6, 10, 8, 6]","[det, nsubj, case, det, nmod, root, case, det,...","[det, nsubj, cop, case, det, root, punct, advc...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,"[0, 1, 0]","[A, person, on, a, horse, jumps, over, a, brok...","[A, person, is, outdoors, ,, on, a, horse, .]","[2, 6, 5, 5, 2, 0, 11, 11, 11, 9, 6, 6]","[2, 4, 4, 0, 4, 8, 8, 4, 4]","[det, nsubj, case, det, nmod, root, case, det,...","[det, nsubj, cop, root, punct, case, det, obl,...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,"[0, 0, 1]","[Children, smiling, and, waving, at, camera]","[They, are, smiling, at, their, parents]","[2, 0, 4, 2, 6, 4]","[3, 3, 0, 6, 6, 3]","[nsubj, root, cc, conj, case, obl]","[nsubj, aux, root, case, nmod:poss, obl]",(ROOT (S (NP (NNS Children)) (VP (VBG smiling)...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,"[0, 1, 0]","[Children, smiling, and, waving, at, camera]","[There, are, children, present]","[2, 0, 4, 2, 6, 4]","[2, 0, 2, 2]","[nsubj, root, cc, conj, case, obl]","[expl, root, nsubj, xcomp]",(ROOT (S (NP (NNS Children)) (VP (VBG smiling)...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NP...
...,...,...,...,...,...,...,...,...,...
550147,"[1, 0, 0]","[Four, dirty, and, barefooted, children, .]","[four, kids, won, awards, for, ', cleanest, fe...","[5, 5, 4, 2, 0, 5]","[2, 3, 0, 3, 8, 8, 8, 4, 8]","[nummod, amod, cc, conj, root, punct]","[nummod, nsubj, root, obj, case, punct, amod, ...",(ROOT (NP (CD Four) (ADJP (JJ dirty) (CC and) ...,(ROOT (S (NP (CD four) (NNS kids)) (VP (VBD wo...
550148,"[0, 0, 1]","[Four, dirty, and, barefooted, children, .]","[four, homeless, children, had, their, shoes, ...","[5, 5, 4, 2, 0, 5]","[3, 3, 4, 0, 6, 4, 4, 13, 13, 11, 13, 13, 4, 4]","[nummod, amod, cc, conj, root, punct]","[nummod, amod, nsubj, root, nmod:poss, obj, xc...",(ROOT (NP (CD Four) (ADJP (JJ dirty) (CC and) ...,(ROOT (S (S (NP (CD four) (JJ homeless) (NNS c...
550149,"[0, 0, 1]","[A, man, is, surfing, in, a, bodysuit, in, bea...","[A, man, in, a, bodysuit, is, competing, in, a...","[2, 4, 4, 0, 7, 7, 4, 11, 11, 11, 4, 4]","[2, 7, 5, 5, 2, 7, 0, 11, 11, 11, 7, 7]","[det, nsubj, aux, root, case, det, obl, case, ...","[det, nsubj, case, det, nmod, aux, root, case,...",(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...
550150,"[1, 0, 0]","[A, man, is, surfing, in, a, bodysuit, in, bea...","[A, man, in, a, business, suit, is, heading, t...","[2, 4, 4, 0, 7, 7, 4, 11, 11, 11, 4, 4]","[2, 8, 6, 6, 6, 2, 8, 0, 12, 12, 12, 8, 8]","[det, nsubj, aux, root, case, det, obl, case, ...","[det, nsubj, case, det, compound, nmod, aux, r...",(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...


In [ ]:
train, w_c_to_idx, c_c_to_idx, dep_lb_to_idx, premises_dict = read_data_pandas_snli(
    train_data, w_c_to_idx, c_c_to_idx, dep_lb_to_idx, {}
)
print("train examples", len(train))

dev, w_c_to_idx, c_c_to_idx, dep_lb_to_idx, premises_dict = read_data_pandas_snli(
    dev_data, w_c_to_idx, c_c_to_idx, dep_lb_to_idx, premises_dict
)

print("dev examples", len(dev))




train examples 549361
dev examples 9842


In [ ]:
print(len(w_c_to_idx), len(c_c_to_idx), len(dep_lb_to_idx))

74 74 49


## Init model

model needs dependency vocab and constituency vocabs

In [ ]:
# np.random.seed(42)
# torch.manual_seed(42)
# torch.cuda.manual_seed(42)
# use_constGCN=False
# use_depGCN=False
# is_syntax_enhanced = use_constGCN or use_depGCN
# model, model_name = initialize_model(768,1, dep_lb_to_idx,w_c_to_idx,c_c_to_idx,device, use_constGCN=use_constGCN, use_depGCN=use_depGCN)
# print(model)

## Init Dataloader

In [ ]:
# # Create tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#re-seed. Init of model creates random parameters.
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Create train dataloader
train_dataset = SNLI_Dataset(train, tokenizer, premises_dict)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=False, collate_fn=lambda batch: get_batch_sup(batch, device, dep_lb_to_idx, use_constGCN, use_depGCN))

# Create validation dataloader
val_dataset = SNLI_Dataset(dev, tokenizer, premises_dict)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda batch: get_batch_sup(batch, device, dep_lb_to_idx, use_constGCN, use_depGCN))

# # Create hard validation dataloader
# val_dataset2 = SNLI_Dataset(dev2, tokenizer, premises_dict)
# val_hard_dataloader = torch.utils.data.DataLoader(val_dataset2, batch_size=32, shuffle=False, collate_fn=lambda batch: get_batch_sup(batch, device, dep_lb_to_idx, use_constGCN, use_depGCN))


## wandb, hyperparameters, optimizers, schedulers

In [ ]:
dataset_name = 'SNLI'
run_name = "SNLI_3runs"
!pip install wandb -Uq
import wandb
wandb.login(key = 'a72edb442b6177a7198f045dee1e6b7c4de8f7a3')
run = wandb.init(project=run_name)

np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
use_constGCN=False
use_depGCN=False
is_syntax_enhanced= use_constGCN or use_depGCN
model, model_name = initialize_model(768, 1, dep_lb_to_idx, w_c_to_idx, c_c_to_idx, device, use_constGCN=use_constGCN, use_depGCN=use_depGCN)
# artifact = run.use_artifact('lucalin/MNLI_3_epochs/cahesyfu.pth:v0', type='model')
# artifact_dir = artifact.download()
# model.load_state_dict(torch.load(artifact_dir+'/'+model_name+'.pth'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lucalin. Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

model name: base
Total parameters = 109484547


In [ ]:
"""## wandb, hyperparameters, optimizers, schedulers"""

# Hyperparameters
batch_size = train_dataloader.batch_size
n_epochs = 2
loss_fn = nn.CrossEntropyLoss()
learning_rate = 3e-5
lr_other = 1e-3

if is_syntax_enhanced:
    optimizer_bert = torch.optim.AdamW(model.bert.parameters(), lr=learning_rate, eps=1e-8)
    scheduler_bert = WarmupLinearSchedule(optimizer_bert,
                                          warmup_steps=int(len(train_dataloader) / 10),
                                          t_total=len(train_dataloader) * n_epochs)
    other_para = [
        {'params': model.hesyfu_layers.parameters()},
        {'params': model.co_attn.parameters()},
        {'params': model.fc.parameters()}
    ]
    optimizer_other = torch.optim.AdamW(other_para, lr=lr_other, eps=1e-8)
    scheduler_other = torch.optim.lr_scheduler.MultiStepLR(optimizer_other, milestones=[1], gamma=0.1, verbose=False)
else:
    optimizer_bert = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps=1e-8)
    scheduler_bert = WarmupLinearSchedule(optimizer_bert,
                                          warmup_steps=int(len(train_dataloader) / 10),
                                          t_total=len(train_dataloader) * n_epochs)
    optimizer_other=None


model_name = model_name
wandb.config.update({
    'model_name': model_name,
    'dataset_name': dataset_name,
    'lr_bert': learning_rate,
    'lr_other': lr_other,
    'epochs': n_epochs,
    'batch_size': batch_size,
    'loss_fn': loss_fn,
    'optimizer': optimizer_bert,
})
wandb.watch(model)

[]

## training pipeline

In [ ]:

train_losses, train_accuracies = [], []

# start training
for epc in range(n_epochs):
    model.to(device)
    print(f'start epoch {epc}')

    # iterate through dataloader
    for i, batch in enumerate(train_dataloader):
        #print(batch[-1])
        loss_batch, accuracy_batch = train_batch( model, batch, loss_fn, optimizer_bert, scheduler_bert, optimizer_other, device, is_syntax_enhanced)
        train_losses.append(loss_batch)
        train_accuracies.append(accuracy_batch)

        if i % 3000 == 0:
            print(f'evaluating batch nr:{i}')
            log_eval_metrics(model, train_losses, train_accuracies, val_dataloader, loss_fn, optimizer_bert, optimizer_other, device, wandb, is_syntax_enhanced)
            train_losses, train_accuracies = [], []

    if is_syntax_enhanced:
        scheduler_other.step()

# save model after last epoch
print('saving model')
model.cpu()
torch.save(model.state_dict(), model_name + '.pth')
artifact = wandb.Artifact(model_name + '.pth', type='model')
artifact.add_file(model_name + '.pth')
run.log_artifact(artifact)

print('last evaluation')
model.to(device)
log_eval_metrics(model, train_losses, train_accuracies, val_dataloader, loss_fn, optimizer_bert, optimizer_other, device, wandb,is_syntax_enhanced)
run.finish()


start epoch 0
evaluating batch nr:0
evaluating batch nr:3000
evaluating batch nr:6000
evaluating batch nr:9000
evaluating batch nr:12000
evaluating batch nr:15000
start epoch 1
evaluating batch nr:0
evaluating batch nr:3000
evaluating batch nr:6000
evaluating batch nr:9000
evaluating batch nr:12000
evaluating batch nr:15000
saving model
last evaluation


In [ ]:
from google.colab import runtime
runtime.unassign()